# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [11]:
import os, collections
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [12]:
oldContents = os.listdir("../_publications/")
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
print("Imported generator `publications.tsv` file:")
publications


Imported generator `publications.tsv` file:


,pub_date,title,venue,excerpt,citation,url,paper_url,url_slug,tags,abstract
0,2016-01-06,Current challenges facing the translation of b...,Frontiers in Cellular Neuroscience,This is a review of the challenges facing the ...,"Murphy, M. D., Guggenmos, D. J., Bundy, D. T.,...",https://www.frontiersin.org/articles/10.3389/f...,https://www.frontiersin.org/articles/10.3389/f...,FrontiersReview,"Review,BCI,Rehabilitation,Brain Computer Inter...",Current research in brain computer interface (...
1,2017-01-09,Comprehensive spatial analysis of the Borrelia...,Journal of Bacteriology,This paper is about the localization of lipopr...,"Dowdell, A. S., Murphy, M. D., Azodi, C., Swan...",https://jb.asm.org/content/jb/199/6/e00658-16....,https://jb.asm.org/content/jb/199/6/e00658-16....,Borrelia,"Borrelia,Burgdorferi,Lipoprotein,Lyme Disease",The Lyme disease spirochete Borrelia burgdorfe...
2,2018-05-27,Assessing Perturbations to Neural Spiking Resp...,International Symposium on Circuits and Systems,This paper is about a proof-of-concept nonline...,"Murphy, M. D., Dunham, C., Nudo, R. J., Guggen...",https://ieeexplore.ieee.org/iel7/8334884/83508...,https://ieeexplore.ieee.org/iel7/8334884/83508...,ISCAS,"ICMS,Stimulation,LFADS,Deep Learning,RNN,Ambul...",This study assesses the feasibility of latent ...
3,2019-10-29,Improving an open-source commercial system to ...,Journal of Neural Engineering,We modified the Intan Stimulation/Recording co...,"Murphy, M., Buccelli, S., Bornat, Y., Bundy, D...",http://m053m716.github.io/files/Murphy_et_al__...,http://m053m716.github.io/files/Murphy_et_al__...,Intan-FSM,"FPGA,Spike Detection,Closed-Loop,ICMS",Objective. Activity-dependent stimulation (ADS...
4,2019-10-30,Chronic stability of single-channel neurophysi...,PLoS ONE,Rats were trained on a 'gross' and 'fine' fore...,"Bundy, D. T., Guggenmos, D. J., Murphy, M. D.,...",https://journals.plos.org/plosone/article/file...,https://journals.plos.org/plosone/article/file...,Bundy,"Rat,Reaching,Motor,Neurophysiology,Spike,LFP,C...",While substantial task-related neural activity...
5,2020-05-14,Differential effects of open-and closed-loop i...,Cerebral cortex,Intracortical microstimulation (ICMS) was appl...,"Averna, A., Pasquale, V., Murphy, M. D., Rogan...",https://academic.oup.com/cercor/article/30/5/2...,https://academic.oup.com/cercor/article/30/5/2...,Averna,"ICMS,ADS,Anesthetized,Rat,Cortex",Intracortical microstimulation can be used suc...
6,2020-11-01,Effects of tDCS on spontaneous spike activity ...,Brain Stimulation,We tested whether transcranial direct-current ...,"Milighetti, S., Sterzi, S., Fregni, F., Hanlon...",https://www.sciencedirect.com/science/article/...,https://www.sciencedirect.com/science/article/...,Millighetti,"tDCS,Rat,Ambulatory,Cortex,FEM,Spike,LFP",The neurophysiological effects of transcranial...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    tags = item.tags.split(",")

    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += 'collection: publications\n'

    md += 'category: publications\n'

    md += 'permalink: /publications/' + html_filename + '/\n'

    md += "redirect_from:\n"

    md += '  - "/publications/' + html_filename + '.html"\n'
    
    md += '  - "/publications/' + html_filename + '"\n'
    
    if len(str(item.excerpt)) > 5:
        md += "excerpt: '" + html_escape(item.excerpt) + "'\n"
    
    md += "date: " + str(item.pub_date) + "\n"
    
    md += "venue: '" + html_escape(item.venue) + "'\n"
    
    if len(str(item.paper_url)) > 5:
        md += "paperurl: '" + item.paper_url + "'\n"
    
    md += "citation: '" + html_escape(item.citation) + "'\n"

    md += "author_profile: true\n"

    ## Add tags in lowercase (they are delimited using "," in the tsv file)
    md += "tags:\n"
    for tag in tags:
        md += "  - " + html_escape(tag.lower()) + "\n"
    
    md += "---\n"
    
    ## Markdown description for individual page
    md += "\n" + html_escape(item.abstract) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.
We can print the contents using the python os `listdir` method to see what has been exported.

In [15]:
newContents = os.listdir("../_publications/")
if collections.Counter(oldContents) == collections.Counter(newContents):
    print("No new files added.")
    print("Existing files:")
    for f in oldContents:
        print("\t" + f)
else:
    print("Generated new files:")
    for f in newContents:
        print("\t" + f)

No new files added.
Existing files:
	2016-01-06-FrontiersReview.md
	2017-01-09-Borrelia.md
	2018-05-27-ISCAS.md
	2019-10-29-Intan-FSM.md
	2019-10-30-Bundy.md
	2020-05-14-Averna.md
	2020-11-01-Millighetti.md
